In [1]:
# Test czy callbacks w ogóle działają:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Test")
output = widgets.Output()

def on_click(b):
    with output:
        print("Button clicked!")

button.on_click(on_click)
display(button)
display(output)

# Kliknij button - powinno wypisać "Button clicked!"

Button(description='Test', style=ButtonStyle())

Output()

In [2]:
import sys
import os
import json
import logging
from google.cloud import secretmanager

#hegemon dependencis
from hegemon.explainability.visualizer import HeatmapGenerator
from hegemon.config.settings import get_settings

/opt/conda/envs/hegemon_env/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:63: UserWarning: `validate_explainability_config` overrides an existing Pydantic `@model_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


In [1]:
%pip install -r streamlit_app/requirements.txt

  Using cached pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.4 MB/s  0:00:00
Using cached pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.8 MB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
Using cached watchdog-6.0.0-py3-non

In [3]:
PROJECT_ID = "dark-data-discovery"
LOCATION = "us-central1"

print(f"📍 GCP Project: {PROJECT_ID}")
print(f"📍 Location: {LOCATION}")

📍 GCP Project: dark-data-discovery
📍 Location: us-central1


In [4]:
def get_secret(project_id, secret_id, version_id="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

try:
    key = get_secret(PROJECT_ID, "ANTHROPIC_API_KEY")
    print(f"✅ ANTHROPIC_API_KEY: {'*' * 10}{key[-4:]}")
except Exception as e:
    print(f"❌ Failed to get ANTHROPIC_API_KEY: {e}")

✅ ANTHROPIC_API_KEY: **********uwAA


In [5]:
# Komórka 1: Setup
# ENABLE EXPLAINABILITY (przez os.environ)
os.environ["HEGEMON_EXPLAINABILITY_ENABLED"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_SEMANTIC_FINGERPRINT"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL"] = "gemini-2.5-flash"  # Vertex AI model

# Setup logging dla Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True
)
logger = logging.getLogger(__name__)

print("✅ Environment configured for Vertex AI")
print(f"   Explainability enabled: {os.environ.get('HEGEMON_EXPLAINABILITY_ENABLED')}")
print(f"   Classifier model: {os.environ.get('HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL')}")

✅ Environment configured for Vertex AI
   Explainability enabled: true
   Classifier model: gemini-2.5-flash


In [6]:
# Komórka 2: Import Hegemon modules
from hegemon.explainability.visualizer import HeatmapGenerator


# Verify settings
settings = get_settings()
print("Explainability Layers:")
print(f"  Layer 6 (Semantic): {settings.explainability_semantic_fingerprint}")
print(f"  Layer 2 (Epistemic): {settings.explainability_epistemic_uncertainty}")


print(f"Configuration:")
print(f"  Katalizator: {settings.katalizator.model}")
print(f"  Sceptyk: {settings.sceptyk.model}")
print(f"  Gubernator: {settings.gubernator.model}")
print(f"  Syntezator: {settings.syntezator.model}")
print(f"\nDebate settings:")
print(f"  Consensus threshold: {settings.debate.consensus_threshold}")
print(f"  Max cycles: {settings.debate.max_cycles}")
# CRITICAL CHECK
if not settings.explainability_enabled:
    print("\n❌ ERROR: Explainability is DISABLED!")
    print("   This means environment variable was set AFTER settings were cached.")
    print("   SOLUTION: Restart kernel and run cells in correct order.")
else:
    print("\n✅ SUCCESS: Explainability is ENABLED!")
    print("   Proceed to next cells.")

print()

2025-10-14 14:01:30,365 - hegemon.config.settings - WARNING - Explainability enabled but no Google API key found. Classifier will fail.
2025-10-14 14:01:30,366 - hegemon.config.settings - INFO - ✅ Sceptyk will use Vertex AI (Application Default Credentials)
2025-10-14 14:01:30,367 - hegemon.config.settings - INFO - ✅ All required authentication validated
2025-10-14 14:01:30,369 - hegemon.config.settings - INFO - ✅ HEGEMON Settings initialized (GCP Project: dark-data-discovery)


Explainability Layers:
  Layer 6 (Semantic): True
  Layer 2 (Epistemic): True
Configuration:
  Katalizator: claude-sonnet-4-5-20250929
  Sceptyk: gemini-2.5-pro
  Gubernator: claude-sonnet-4-5-20250929
  Syntezator: claude-opus-4-1-20250805

Debate settings:
  Consensus threshold: 0.7
  Max cycles: 5

✅ SUCCESS: Explainability is ENABLED!
   Proceed to next cells.



In [7]:
# Zdefiniuj swoją misję
MISSION = """
Design a comprehensive AI strategy for a mid-size e-commerce company.

Context:
- Company: 150 employees, $50M annual revenue
- Current tech stack: Shopify + basic analytics
- Budget: $500K annually for AI initiatives
- Timeline: 12 months

Requirements:
1. Improve customer experience (personalization, recommendations)
2. Reduce operational costs (automation, optimization)
3. GDPR compliant
4. Limited ML expertise in-house

Deliverables needed:
- Technology stack recommendations
- Phased implementation roadmap (Q1-Q4)
- Team structure and hiring plan
- Risk analysis
- Success metrics (KPIs)
"""

print("📋 Mission defined:")
print(MISSION)

📋 Mission defined:

Design a comprehensive AI strategy for a mid-size e-commerce company.

Context:
- Company: 150 employees, $50M annual revenue
- Current tech stack: Shopify + basic analytics
- Budget: $500K annually for AI initiatives
- Timeline: 12 months

Requirements:
1. Improve customer experience (personalization, recommendations)
2. Reduce operational costs (automation, optimization)
3. GDPR compliant
4. Limited ML expertise in-house

Deliverables needed:
- Technology stack recommendations
- Phased implementation roadmap (Q1-Q4)
- Team structure and hiring plan
- Risk analysis
- Success metrics (KPIs)



In [8]:
# Wybierz tryb
MODE = "reviewer"  # Zmień na "observer" lub "collaborator"

print(f"✅ Selected mode: {MODE}")
print(f"\nExpected checkpoints:")
if MODE == "observer":
    print("  - PRE_SYNTHESIS (1x przed syntezą)")
elif MODE == "reviewer":
    print("  - POST_THESIS (po każdym Katalizatorze)")
    print("  - POST_EVALUATION (po Gubernatorze)")
    print("  - PRE_SYNTHESIS (przed Syntetatorem)")
else:
    print("  - All checkpoints + detailed guidance")

✅ Selected mode: reviewer

Expected checkpoints:
  - POST_THESIS (po każdym Katalizatorze)
  - POST_EVALUATION (po Gubernatorze)
  - PRE_SYNTHESIS (przed Syntetatorem)


In [9]:
# Główne importy HITL
from hegemon.graph_hitl_v3 import (
    create_hegemon_graph_hitl_v3,
    run_debate_hitl_v3,
)

In [10]:
# 🚀 START DEBATE!
print("="*80)
print("🤖 Starting HITL Debate...")
print("="*80)
# Run debate (INTERACTIVE - będzie czekać na Twój input!)
final_state = run_debate_hitl_v3(
    mission=MISSION,
    intervention_mode=MODE,
)

print("\n" + "="*80)
print("✅ Debate Complete!")
print("="*80)

2025-10-14 14:31:32,177 - hegemon.agents - INFO - ⚔️ SCEPTYK (google/gemini-2.5-pro via Vertex AI): Generating Antithesis (Cycle 1)
2025-10-14 14:31:32,179 - langchain_google_vertexai.chat_models - WARNING - Unexpected argument 'timeout' provided to ChatVertexAI.
/home/jupyter/olga_zydziak/version_beta/Folder/hegemon_mvp/hegemon/agents.py:301: UserWarning: WARNING! timeout is not default parameter.
                timeout was transferred to model_kwargs.
                Please confirm that timeout is what you intended.
  llm = get_llm_for_agent("Sceptyk")
2025-10-14 14:31:58,023 - hegemon.agents - INFO - ✅ SCEPTYK: Antithesis generated (1935 chars)
2025-10-14 14:31:58,024 - hegemon.explainability.collector - INFO - 🔍 Collecting explainability for Sceptyk (Cycle 1)
2025-10-14 14:32:07,507 - hegemon.explainability.classifier - INFO - ✅ Classification successful: 9482ms, top_concept=('caution', 0.8)
2025-10-14 14:32:13,285 - hegemon.explainability.epistemic - INFO - Claim extraction succe

OverloadedError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}, 'request_id': None}

In [ ]:
# Summary statistics
print("📊 DEBATE SUMMARY")
print("="*80)
print(f"Total cycles: {final_state.cycle_count}")
print(f"Total contributions: {len(final_state.contributions)}")
print(f"Human feedbacks provided: {len(final_state.human_feedback)}")
print(f"Revisions requested: {final_state.revision_count}")
print(f"Final consensus score: {final_state.current_consensus_score:.2f}")
print()

In [ ]:
# Feedback breakdown
if final_state.human_feedback:
    print("📝 YOUR FEEDBACK HISTORY")
    print("="*80)
    
    for i, feedback in enumerate(final_state.human_feedback, 1):
        print(f"\n{i}. Checkpoint: {feedback.checkpoint.value}")
        print(f"   Decision: {feedback.decision.value}")
        print(f"   Timestamp: {feedback.timestamp.strftime('%H:%M:%S')}")
        
        if feedback.guidance:
            print(f"   Your guidance: {feedback.guidance[:100]}...")
        
        if feedback.priority_claims:
            print(f"   Priority claims: {len(feedback.priority_claims)}")
        
        if feedback.flagged_concerns:
            print(f"   Flagged concerns: {len(feedback.flagged_concerns)}")

In [ ]:
# Final Plan overview
if final_state.final_plan:
    print("\n📋 FINAL PLAN OVERVIEW")
    print("="*80)
    
    plan = final_state.final_plan
    
    print(f"\nMission: {plan.mission_overview[:200]}...")
    print(f"\nRequired Agents: {len(plan.required_agents)}")
    for agent in plan.required_agents[:5]:  # Show first 5
        print(f"  - {agent.role}: {', '.join(agent.skills[:3])}")
    
    print(f"\nWorkflow Steps: {len(plan.workflow)}")
    for step in plan.workflow[:5]:  # Show first 5
        print(f"  {step.step_number}. {step.description[:60]}...")
        print(f"     Agent: {step.assigned_agent} | Duration: {step.estimated_duration}")
    
    print(f"\nTotal workflow duration: {plan.total_estimated_duration}")
    print(f"Risk score: {plan.overall_risk_score:.2f}")
else:
    print("\n⚠️ No final plan generated (debate may have been rejected)")

In [ ]:
# Save to JSON
import json
from datetime import datetime

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"hegemon_hitl_{MODE}_{timestamp}.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(
        final_state.model_dump(mode='json'),
        f,
        indent=2,
        ensure_ascii=False,
    )

print(f"💾 Results saved to: {output_file}")
print(f"   File size: {output_file.stat().st_size / 1024:.1f} KB")

In [ ]:
# Show all agent contributions
print("🔍 FULL DEBATE HISTORY")
print("="*80)

for contrib in final_state.contributions:
    print(f"\n{'='*80}")
    print(f"Agent: {contrib.agent_id} | Type: {contrib.type} | Cycle: {contrib.cycle}")
    print(f"{'='*80}")
    print(f"\n{contrib.content[:500]}...")
    print(f"\nRationale: {contrib.rationale[:200]}...")

In [ ]:
# Calculate metrics
from collections import Counter

print("📈 PERFORMANCE METRICS")
print("="*80)

# Intervention metrics
intervention_rate = len(final_state.human_feedback) / max(final_state.cycle_count, 1)
print(f"\nIntervention rate: {intervention_rate:.2f} feedbacks/cycle")

# Decision breakdown
if final_state.human_feedback:
    decisions = Counter(f.decision.value for f in final_state.human_feedback)
    total = len(final_state.human_feedback)
    
    print(f"\nDecision breakdown:")
    for decision, count in decisions.items():
        percentage = (count / total) * 100
        print(f"  {decision}: {count} ({percentage:.1f}%)")
    
    approval_rate = decisions.get('approve', 0) / total
    print(f"\nApproval rate: {approval_rate:.1%}")

# Revision metrics
revision_rate = final_state.revision_count / max(final_state.cycle_count, 1)
print(f"\nRevision rate: {revision_rate:.2f} revisions/cycle")

# Quality indicator
quality_score = final_state.current_consensus_score * approval_rate
print(f"\nQuality score: {quality_score:.2f} (consensus × approval rate)")